# Battle of the Neighborhoods - DRAFT

## Research: List top world cities by population

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [32]:
#Parse the list of top world cities by popuylation from Wikipedia and clean data into pandas dataframe
url = 'https://en.wikipedia.org/wiki/List_of_cities_proper_by_population'
sauce = requests.get(url).text
soup = BeautifulSoup(sauce, 'lxml')
tables = [[[td.get_text(strip=True)
           for td in tr.find_all('td')]
           for tr in table.find_all('tr')]
         for table in soup.find_all('table')]
world_cities = tables[1]
df = pd.DataFrame(world_cities, 
                 columns =['Rank', 'City', 'Image', 'Population', 'Definition', 'Total area (km2)', 'Population density (/km2)', 'Country'])
#Remove unnecessary row with no values
df.drop(df.index[0], inplace=True)
#Remove unnecessary columns with no useful information
df.drop(columns = ['Image', 'Definition'], inplace=True)
#Remove [references to footnotes] from dataframe using regex
for column in df.columns:
    df[column] = df[column].str.replace(r"\[.*]","")
#Remove commas from columns containig numbers using regex and convert type to numeric
NumberColumns = ['Rank', 'Population', 'Total area (km2)', 'Population density (/km2)']
for column in NumberColumns:
    df[column] = pd.to_numeric(df[column].str.replace(",",""))
print('Dataframe shape: ',df.shape)
df.head()

Dataframe shape:  (86, 6)


,Rank,City,Population,Total area (km2),Population density (/km2),Country
1,1,Chongqing,30165500,82403.0,366,China
2,2,Shanghai,24183300,6340.5,3814,China
3,3,Beijing,21707000,16411.0,1267,China
4,4,Istanbul,15029231,5196.0,2893,Turkey
5,5,Karachi,14910352,3780.0,3944,Pakistan


In [33]:
#Find world cities coordinates


Rank                           int64
City                          object
Population                     int64
Total area (km2)             float64
Population density (/km2)      int64
Country                       object
dtype: object